In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import openeo
import shapely.geometry
from shapely.geometry import Polygon, box, MultiPolygon
from shapely.ops import unary_union
from ipyleaflet import Map, basemaps, DrawControl, GeoData
from ipywidgets import Button, Output, DatePicker
from IPython.display import display
import LCE
import BCET
from openeo.processes import ProcessBuilder

# Create a map
m = Map(center=(25, 50), zoom=5, basemap=basemaps.OpenStreetMap.Mapnik)

# Create a draw control
draw_control = DrawControl(polygon={'shapeOptions': {'color': '#0000FF'}})
m.add_control(draw_control)

# Display the map
display(m)

# Create an output widget to display results
output = Output()
display(output)

def tcc(polygon):
    # Create date picker widgets for the start and end time
    start_time_picker = DatePicker(description='Start Time')
    end_time_picker = DatePicker(description='End Time')

    # Display the date pickers
    display(start_time_picker)
    display(end_time_picker)

    # Create a button for the user to click when they have picked the dates
    button = Button(description='Done')
    display(button)

    # Define a function to execute when the button is clicked
    def on_button_clicked(b):
        # Get the picked dates
        start_time = start_time_picker.value
        end_time = end_time_picker.value
        connection = openeo.connect(url='openeo.dataspace.copernicus.eu').authenticate_oidc()

        # Get the bounding box of the polygon
        bbox = polygon.bounds

        cube = (
        connection.load_collection(
            "SENTINEL2_L2A",
            temporal_extent=[start_time, end_time],
            spatial_extent=dict(zip(["west", "south", "east", "north"], bbox)),
            bands=["B02", "B03", "B04"]))
        
        cube_b234_reduced = cube.mean_time()
        

        def scale_function(x: ProcessBuilder):
            return x.linear_scale_range(0, 6000, 0, 255)

        cube_b234_reduced_lin = cube_b234_reduced.apply(scale_function)

        res = cube_b234_reduced_lin.save_result(format="GTIFF", options={
            "red": "B4",
            "green": "B3",
            "blue": "B2"
        })

        res.download('tcc_example.tiff', format='GTIFF')
        BCET.plot_result_bcet('tcc_example.tiff')
        LCE.plot_result('tcc_example.tiff', 10)

    button.on_click(on_button_clicked)

# Function to execute when a new shape is drawn
def handle_draw(self, action, geo_json):
    # Get the coordinates of the polygon
    coords = geo_json['geometry']['coordinates'][0]
    polygon = Polygon(coords)

    # Define the size of your square sections (in degrees)
    square_size = 0.005

    # Create a grid of squares within your area of interest
    squares = []
    for x in np.arange(min([coord[0] for coord in coords]), max([coord[0] for coord in coords]), square_size):
        for y in np.arange(min([coord[1] for coord in coords]), max([coord[1] for coord in coords]), square_size):
            square = box(x, y, x+square_size, y+square_size)
            if polygon.intersects(square):
                squares.append(square)

    # Create a GeoDataFrame for the squares
    squares_gdf = gpd.GeoDataFrame(gpd.GeoSeries(squares), columns=['geometry'])

    # Display the squares on the map
    m.add_layer(GeoData(geo_dataframe=squares_gdf))

    # Create a MultiPolygon object from the squares
    multipolygon = MultiPolygon(squares)

    # Create a single polygon that represents the outline of the area
    outline = unary_union(multipolygon)

    # Check if the outline is a MultiPolygon
    if outline.geom_type == 'MultiPolygon':
        # If it is, then iterate over each Polygon and extract the coordinates
        outline_coords = [list(p.exterior.coords) for p in outline.geoms]
        # Flatten the list of coordinates
        outline_coords = [coord for sublist in outline_coords for coord in sublist]
    else:
        # If it's not a MultiPolygon (i.e., it's a single Polygon), then extract the coordinates as before
        outline_coords = list(outline.exterior.coords)

    # Add the first coordinate to the end to close the shape
    outline_coords.append(outline_coords[0])

    # Convert the coordinates to a DataFrame
    outline_df = pd.DataFrame(outline_coords, columns=['longitude', 'latitude'])

    # Save the DataFrame to a .csv file
    outline_df.to_csv('main_shape.csv', index=False)

    tcc(polygon)

# Attach the function to the draw control
draw_control.on_draw(handle_draw)
